Final Cleaned Up code

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/bintefatimatuzzahra28/ML_Project/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.9 MB/s eta 

In [3]:
import getpass
import os

if not os.getenv("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

Enter your Cohere API key: ··········


In [4]:
import getpass
import os

if not os.getenv("GoogleVertex_API_KEY"):
    os.environ["GoogleVertex_API_KEY"] = getpass.getpass("Enter your Google Vertex API key: ")

Enter your Google Vertex API key: ··········


In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/bintefatimatuzzahra28/ML_Project/mymlproject-444721-140c4261cfb8.json"

In [6]:
import os
import json
import numpy as np
import faiss
import streamlit as st
from langchain.document_loaders import PyPDFLoader, TextLoader
from tempfile import NamedTemporaryFile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_vertexai import ChatVertexAI
from tenacity import retry, stop_after_attempt, wait_exponential
from pyngrok import ngrok

ngrok.set_auth_token("2po1TCRu94I8GFTrAfJ4wkhmbKj_7CkEpn4edG2oAWV2t9zQa")

# Token estimation function
def estimate_tokens(text):
    words = text.split()
    return int(len(words) / 0.75)  # Rough token estimate per document


# Process Uploaded Files
def process_uploaded_files(uploaded_files):
    """Process and load documents from uploaded files."""
    document_list = []
    for uploaded_file in uploaded_files:
        # Save the uploaded file to a temporary file
        file_extension = os.path.splitext(uploaded_file.name)[1].lower()

        with NamedTemporaryFile(delete=False, suffix=file_extension) as tmp_file:
            tmp_file.write(uploaded_file.getvalue())
            temp_file_path = tmp_file.name

        # Choose loader based on file type
        if file_extension == ".pdf":
            loader = PyPDFLoader(temp_file_path)
        elif file_extension == ".txt":
            loader = TextLoader(temp_file_path, encoding="utf-8")  # Use UTF-8 encoding for text files
        else:
            st.warning(f"Unsupported file format: {file_extension}. Skipping {uploaded_file.name}")
            continue

        # Load documents and extend the document list
        try:
            documents = loader.load()
            document_list.extend(documents)
        except Exception as e:
            st.error(f"Error loading {uploaded_file.name}: {e}")
            continue

        # Optionally clean up the temporary file after processing
        os.remove(temp_file_path)

    return document_list


# Split Documents into Chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20, max_tokens=2000):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(documents)

    # Ensure chunks do not exceed token limit
    valid_chunks = []
    for chunk in chunks:
        token_count = estimate_tokens(chunk.page_content)
        if token_count <= max_tokens:
            valid_chunks.append(chunk)
        else:
            st.warning(f"Skipping chunk due to token limit: {chunk.page_content[:100]}... (tokens: {token_count})")

    return valid_chunks


# Create the Cohere Embedding Model
embeddings = CohereEmbeddings(model="embed-english-v3.0")


# Implement Retry Logic for Embeddings
@retry(stop=stop_after_attempt(5), wait=wait_exponential(min=1, max=10))
def embed_documents_with_retry(texts, embeddings):
    return embeddings.embed_documents(texts)


# Create FAISS Vectorstore from Docs
def create_faiss_vectorstore_from_docs(docs, embeddings, faiss_index_path, metadata_path):
    texts = [doc.page_content for doc in docs]

    # Batch embedding to prevent rate limit issues
    embeddings_matrix = []
    batch_size = 100  # Adjust based on rate limit
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        embeddings_matrix.extend(embed_documents_with_retry(batch, embeddings))

    embeddings_array = np.array(embeddings_matrix).astype('float32')
    dimension = embeddings_array.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_array)

    # Overwrite FAISS index if it exists
    if os.path.exists(faiss_index_path):
        os.remove(faiss_index_path)
    faiss.write_index(index, faiss_index_path)

    # Save updated metadata
    metadata = [{'doc_id': i, 'content': doc.page_content} for i, doc in enumerate(docs)]
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f)

    return index


# Perform Retrieval and Generate Answer
def search_similar_docs_with_faiss_and_generate_answer(query, index, metadata, embeddings, model, k=3, max_context_tokens=2000):
    """Retrieve and synthesize information from multiple documents."""
    query_embedding = embeddings.embed_query(query)
    query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)

    # Perform FAISS search
    _, indices = index.search(query_embedding, k)

    if indices[0].size > 0 and np.any(indices[0] != -1):  # Ensure valid retrieval
        retrieved_docs = [metadata[i] for i in indices[0] if i != -1]
        st.write(f"Retrieved {len(retrieved_docs)} document(s) for the query:")
        #for i, doc in enumerate(retrieved_docs):                                   #for turning on data retrieval
        #    st.write(f"Document {i+1} Snippet: {doc['content'][:200]}...")  # Display snippet

        # Concatenate context for the LLM
        context = "\n---\n".join([f"Document {i+1}:\n{doc['content']}" for i, doc in enumerate(retrieved_docs)])

        # Summarize context if it exceeds token limit
        if len(context.split()) > max_context_tokens:
            st.write("Context too large; summarizing...")
            context = summarize_long_context(retrieved_docs, model, max_context_tokens)

        if not context.strip():
            st.write("No sufficient context retrieved to answer the query.")
            return "I cannot determine this from the provided information."

        # Generate the answer
        #st.write(f"Context passed to LLM: {context[:500]}...")  # Display first 500 chars for debugging
        answer = generate_answer_with_llm(query, retrieved_docs, model)
        st.write(f"### Generated Answer: {answer}")
    else:
        st.write("No relevant documents found for the query.")
        return "I cannot determine this from the provided information."



# Generate Answer Using LLM
def generate_answer_with_llm(query, retrieved_docs, model):
    context = "\n---\n".join([f"Document {i+1}:\n{doc['content']}" for i, doc in enumerate(retrieved_docs)])

    prompt = PromptTemplate(
        input_variables=["question", "context"],
        template="""You are a helpful and advanced assistant designed to process information from provided documents and you can reason across multiple documents.
                    Your task is to answer the question based on the provided documents. If the answer cannot be determined from the documents, respond with "I cannot determine this from the provided information."

        Context:
        {context}

        Question:
        {question}

        Answer:"""
    )

    llm_chain = LLMChain(prompt=prompt, llm=model)
    answer = llm_chain.run({"question": query, "context": context})
    return answer


# Summarize Long Context to Fit Token Limit
def summarize_long_context(retrieved_docs, model, max_context_tokens=2000):
    """Summarize documents to fit within the token limit."""
    summaries = []
    prompt = PromptTemplate(
        input_variables=["content"],
        template="""Summarize the following document to capture the key points in 200 words or less:

        {content}

        Summary:"""
    )

    llm_chain = LLMChain(prompt=prompt, llm=model)

    for doc in retrieved_docs:
        summary = llm_chain.run({"content": doc['content']})
        summaries.append(summary.strip())

    # Combine summaries, ensuring they fit within the token limit
    combined_summary = "\n---\n".join(summaries)
    return combined_summary[:max_context_tokens]

# Process Uploaded Files and Update FAISS Index
def update_index_with_new_files(uploaded_files, faiss_index_path, metadata_path, embeddings):
    if uploaded_files:
        # Process newly uploaded documents
        new_docs = process_uploaded_files(uploaded_files)
        new_docs = split_docs(new_docs, chunk_size=1000, chunk_overlap=20, max_tokens=2000)

        if new_docs:
            # Load existing index and metadata if they exist
            if os.path.exists(faiss_index_path) and os.path.exists(metadata_path):
                st.write("Updating existing FAISS index...")
                existing_index = faiss.read_index(faiss_index_path)
                with open(metadata_path, 'r') as f:
                    existing_metadata = json.load(f)
            else:
                st.write("Creating a new FAISS index...")
                existing_index = None
                existing_metadata = []

            # Embed new documents
            new_texts = [doc.page_content for doc in new_docs]
            new_embeddings = []
            for i in range(0, len(new_texts), 100):  # Batch embedding
                batch = new_texts[i:i + 100]
                new_embeddings.extend(embed_documents_with_retry(batch, embeddings))

            new_embeddings_array = np.array(new_embeddings).astype('float32')

            # Create or update FAISS index
            if existing_index:
                existing_index.add(new_embeddings_array)
                metadata = existing_metadata + [{'doc_id': len(existing_metadata) + i, 'content': doc.page_content}
                                                 for i, doc in enumerate(new_docs)]
            else:
                dimension = new_embeddings_array.shape[1]
                existing_index = faiss.IndexFlatL2(dimension)
                existing_index.add(new_embeddings_array)
                metadata = [{'doc_id': i, 'content': doc.page_content} for i, doc in enumerate(new_docs)]

            # Save the updated FAISS index and metadata
            faiss.write_index(existing_index, faiss_index_path)
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f)

            return existing_index, metadata
    return None, None


# Load FAISS Index and Metadata
def load_faiss_index(faiss_index_path):
    try:
        index = faiss.read_index(faiss_index_path)
        return index
    except Exception as e:
        st.write(f"Error loading FAISS index: {e}")
        return None

def load_metadata(metadata_path):
    try:
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        return metadata
    except Exception as e:
        st.write(f"Error loading metadata: {e}")
        return []


# Main Streamlit Interface
def main():
    st.title("Document-based Question Answering System")

    # Upload Documents
    uploaded_files = st.file_uploader(
        "Upload your documents (.pdf or .txt):",
        type=["pdf", "txt"],
        accept_multiple_files=True,
    )

    # FAISS index and metadata paths (hidden)
    faiss_index_path = "/content/drive/MyDrive/bintefatimatuzzahra28/ML_Project/faiss_metadata/faiss_index.index"
    metadata_path = "/content/drive/MyDrive/bintefatimatuzzahra28/ML_Project/faiss_metadata/metadata.json"

    # Initialize Vertex AI model
    model = ChatVertexAI(model="gemini-1.5-flash", project_id="mymlproject-444721")

    # Check if FAISS index exists and load or create
    if os.path.exists(faiss_index_path) and os.path.exists(metadata_path):
        st.write("FAISS index exists. Loading FAISS index...")
        metadata = load_metadata(metadata_path)
        index = load_faiss_index(faiss_index_path)
    else:
        st.write("No FAISS index found. Please upload documents to create the index.")
        metadata, index = None, None

    # Update FAISS index and metadata if new files are uploaded
    new_index, new_metadata = update_index_with_new_files(
        uploaded_files, faiss_index_path, metadata_path, embeddings
    )
    if new_index and new_metadata:
        index = new_index
        metadata = new_metadata

    # Query System
    query = st.text_input("Enter your question:")

    if query:
        if index and metadata:
            search_similar_docs_with_faiss_and_generate_answer(query, index, metadata, embeddings, model)
        else:
            st.write("No documents available for querying. Please upload files.")

if __name__ == "__main__":
    main()


2024-12-15 18:41:28.663 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.065 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-15 18:41:29.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.069 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-15 18:41:29.078 Thread 'MainThread': mi

In [7]:
%cd "/content/drive/MyDrive/bintefatimatuzzahra28/ML_Project/"

/content/drive/MyDrive/bintefatimatuzzahra28/ML_Project


In [8]:
from pyngrok import ngrok

# Start Streamlit in the background
!streamlit run finalcode.py &>/dev/null &
#!streamlit run txtloader_w_new_prompt.py &>/dev/null &
#!streamlit run New_w_txtloader.py &>/dev/null &
#!streamlit run metadata_faiss_index_fixed_wsettingsapply.py &>/dev/null &
#!streamlit run app.py &>/dev/null &
#!streamlit run app.py

# Set up an ngrok tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://94f4-34-125-96-178.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!lsof -i:8501

In [ ]:
!kill -9 7620